In [1]:
# https://www.kaggle.com/competitions/benin-national-ai-olympiad-selection

import os
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pl.read_csv("/kaggle/input/benin-national-ai-olympiad-selection/train.csv")
test = pl.read_csv("/kaggle/input/benin-national-ai-olympiad-selection/test.csv")
subm = pl.read_csv("/kaggle/input/benin-national-ai-olympiad-selection/sample_submission.csv")

train.shape, test.shape, subm.shape

((1000000, 287), (500000, 286), (500000, 2))

In [3]:
# train = (
#     train.group_by('target', maintain_order=True)
#     .map_groups(lambda df: df.sample(fraction=0.1, with_replacement=False, seed=42))
# )

# train.shape

In [4]:
train.head()

id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,…,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,0.205979,0.410993,0.176775,0.223581,0.423543,0.47614,0.41359,0.612021,0.534873,0.147295,0.0261771,0.106613,0.200924,0.713191,0.15575,0.557335,0.341702,0.28572,0.230396,0.203957,0.509588,0.706972,1,0.007793,0.247765,0.26375,0.259555,0.23173,0.138379,0.197824,0.0543923,0.194153,0.2815,0.0348178,0.0253337,0.114432,…,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1
1,0.181004,0.473119,0.0117337,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,0.493411,0.107277,0.231828,0.45715,0.3952,0.617088,0.459358,0.209225,0.201098,0.199383,0.366578,0.585788,1,0.285311,0.400367,0.162493,0.249365,0.14116,0.133688,0.247906,0.139251,0.216444,0.109674,0.0330182,0.0174584,0.189336,…,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1
2,0.182583,0.307431,0.32595,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,0.286813,0.139532,0.107222,0.247791,0.631949,0.347463,0.642173,0.257763,0.162548,0.327377,0.193583,0.49544,0.636742,0,0.007133,0.309747,0.221081,0.28481,0.230828,0.138271,0.199742,0.0604075,0.146746,0.208131,0.0359773,0.0226313,0.113542,…,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1
3,0.18024,0.494592,0.0083673,0.22358,0.760618,0.439211,0.432055,0.776147,0.483958,0.260886,0.147122,0.105433,0.287755,0.455777,0.247971,0.616628,0.335907,0.337025,0.239127,0.176163,0.538269,0.706468,1,0.008829,0.353799,0.219977,0.266858,0.145617,0.13859,0.234925,0.0598173,0.140886,0.205023,0.319312,0.009972,0.112291,…,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
4,0.177172,0.495513,0.0142631,0.548819,0.625396,0.562493,0.117158,0.561255,0.0771147,0.158321,0.260239,0.102561,0.265285,0.503776,0.269776,0.545945,0.319548,0.278538,0.214922,0.200239,0.534551,0.728652,0,0.00484,0.323546,0.166292,0.285516,0.208651,0.200394,0.198155,0.414729,0.251997,0.193405,0.0344898,0.0164647,0.197312,…,1,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1


In [5]:
(
    train.with_columns(pl.col(train.columns).is_nan())
    .select(pl.all().sum())
    .select(pl.sum_horizontal(pl.all()))
    .item()
)

0

In [6]:
(
    test.with_columns(pl.col(test.columns).is_nan())
    .select(pl.all().sum())
    .select(pl.sum_horizontal(pl.all()))
    .item()
)

0

In [7]:
from sklearn.model_selection import train_test_split

features = [c for c in train.columns if c not in ['id', 'target']]
target_col = 'target'
X, y = train.select(features).to_numpy(), train.select(target_col).to_numpy().ravel()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train.shape, X_valid.shape

((900000, 285), (100000, 285))

In [8]:
from catboost import CatBoostClassifier

params = {
    'iterations': 1000,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'metric_period': 100,
    'max_depth': 4,
    'random_state': 42
}

model = CatBoostClassifier(**params)

In [9]:
%%time

model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

Learning rate set to 0.170062
0:	test: 0.8154740	best: 0.8154740 (0)	total: 507ms	remaining: 8m 26s
100:	test: 0.8505913	best: 0.8505913 (100)	total: 34.2s	remaining: 5m 4s
200:	test: 0.8542722	best: 0.8542722 (200)	total: 1m 5s	remaining: 4m 21s
300:	test: 0.8555632	best: 0.8555632 (300)	total: 1m 36s	remaining: 3m 45s
400:	test: 0.8562178	best: 0.8562178 (400)	total: 2m 7s	remaining: 3m 10s
500:	test: 0.8565239	best: 0.8565239 (500)	total: 2m 37s	remaining: 2m 36s
600:	test: 0.8566082	best: 0.8566082 (600)	total: 3m 7s	remaining: 2m 4s
700:	test: 0.8566567	best: 0.8566567 (700)	total: 3m 37s	remaining: 1m 32s
800:	test: 0.8566576	best: 0.8566576 (800)	total: 4m 6s	remaining: 1m 1s
900:	test: 0.8566527	best: 0.8566576 (800)	total: 4m 35s	remaining: 30.3s
999:	test: 0.8566378	best: 0.8566576 (800)	total: 5m 4s	remaining: 0us

bestTest = 0.8566575999
bestIteration = 800

Shrink model to first 801 iterations.
CPU times: user 22min 2s, sys: 26.4 s, total: 22min 28s
Wall time: 7min 8s


In [10]:
from sklearn.metrics import roc_auc_score

y_pred = model.predict_proba(X_valid)[:, 1]

score = roc_auc_score(y_valid, y_pred)

print(f'Score: {score:.5f}')

Score: 0.85666


In [11]:
X_test = test.select(features).to_numpy()

y_pred = model.predict_proba(X_test)[:, 1]

subm = subm.with_columns(
    pl.Series("target", y_pred)
)

subm.write_csv("submission.csv")

subm.head()

id,target
i64,f64
1000000,0.760806
1000001,0.231393
1000002,0.908216
1000003,0.864314
1000004,0.231164
